#  Toronto Data and Clustering

### Importing packages

In [1]:
import numpy as np 
import pandas as pd 

import requests 
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium 

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Data collection from Wikipedia website.

Used read_html to gain access to the specific table with the matched word "Postcode". Then usd the df[0] to convert it to pandas DataFrame. Dropped the null results for 'Borough' and changed the name of the column. Also changed the null value.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
        
df=pd.read_html(url, match='Postcode', header=0, na_values = 'Not assigned', keep_default_na=False)

df=df[0]

df = df.dropna(axis=0, how= 'any', subset= ['Borough'], inplace=False)

df.rename(columns = {'Postcode': 'Postal Code'}, inplace = True)
df.fillna(value= "Queen's Park", axis=0, inplace = True)
df.head(2)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village


In [3]:
df.isnull().sum()

Postal Code      0
Borough          0
Neighbourhood    0
dtype: int64

Grouped the data and made sure the Neighbourhoods were merged with a comma in between.

In [4]:
df2 = df.groupby(by= ['Postal Code', 'Borough'], as_index=False).agg(', '.join)


In [5]:
df2.head(2)

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"


In [6]:
print('The shape of the DataFrame is: ', df2.shape)
df2.head(6)

The shape of the DataFrame is:  (103, 3)


,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village


## PART TWO

This part is to find longitude and latitude for each postcode.

In [7]:
long_lat_csv = 'http://cocl.us/Geospatial_data'

data = pd.read_csv(long_lat_csv, sep=',', index_col = 'Postal Code')
long_lat_df= pd.DataFrame(data = data)


In [10]:
print(long_lat_df.shape)
long_lat_df.head()

(103, 2)


,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [11]:
df2.set_index('Postal Code')

,Borough,Neighbourhood
Postal Code,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"


Connected both DF to one single one.

In [12]:
joined_df = df2.join(long_lat_df, on='Postal Code', sort=False)
joined_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
joined_df.shape

(103, 5)

## PART THREE - Clustering

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.

to generate maps to visualize your neighborhoods and how they cluster together.

Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)

Map of Toronto with the neighbourhoods and boroughs as markers

In [15]:
latitude= 43.7
longitude=-79.3832

In [16]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, Borough, Neighborhood in zip(joined_df['Latitude'], joined_df['Longitude'], joined_df['Borough'], joined_df['Neighbourhood']):
    label = '{}, {}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  

map_newyork

This part  is getting ready for Clustering and running clustering sunction:

In [18]:
# set number of clusters
kclusters = 5


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(long_lat_df)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 2, 0, 2], dtype=int32)

In [20]:
# add clustering labels
joined_df.insert(0, 'Cluster Labels', kmeans.labels_)

t_merged = joined_df


t_merged.head() # check the last columns!


,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,0,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,0,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,0,M1G,Scarborough,Woburn,43.770992,-79.216917
4,0,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [21]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(t_merged['Latitude'], t_merged['Longitude'], t_merged['Neighbourhood'], t_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

This is the map of the Clustered points of neighbourhoods in Toronto!